In [102]:
!pip install nb_black -q
!pip install pdfminer.six -q
!pip install urllib3 -q
!pip install bs4 -q

In [34]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


### Aluno Marco Antonio Moreira Carujo

# Atividade 1: Coleta de Documentos em PDF
Atividade: Baixar arquivos PDFs da Web e armazená-los como arquivo de texto.

Site: 2019 - Anais do IV Congresso sobre Tecnologias na Educação (https://sol.sbc.org.br/index.php/ctrle)

In [35]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from joblib import Parallel, delayed

Dentro do site temos dois links um comprido onde mostra apenas os ultimos artigos e um completo onde temos todos os artigos (75 artigos) do ano de 2019.

Podemos baixar ele e ir na tag **\<a>** buscar o href de cada um. Dentro do href apenas mudando a parametro/rota view para download podemos baixar cada um dos PDF's.
    
A tag a possuí uma classe chama **obj_galley_link pdf** o que facilita o encontro da mesma.

In [36]:
response = urlopen("https://sol.sbc.org.br/index.php/ctrle")  # Comprimido
response = urlopen("https://sol.sbc.org.br/index.php/ctrle/issue/view/486")  # Completo
html = response.read()

soup = BeautifulSoup(html, "html.parser")
elements = soup.find_all("a", class_="obj_galley_link pdf")
authors = soup.find_all("div", class_="authors")
title = soup.find_all("div", class_="title")
print("Quantos artigos temos no site?", len(elements), "artigos.")

Quantos artigos temos no site? 75 artigos.


Apenas manipulações de objetos e tranformações de Strings para gerar as seguintes listas:
- lista de links de view
- lista de links de download
- lista de links de nomes de arquivos para ser tanto .pdf quanto .txt

In [37]:
links_to_view = [el["href"] for el in elements]
links_to_view[:2]

['https://sol.sbc.org.br/index.php/ctrle/article/view/8870/8771',
 'https://sol.sbc.org.br/index.php/ctrle/article/view/8871/8772']

In [38]:
links_to_download = [el.replace("view", "download") for el in links_to_view]
links_to_download[:2]

['https://sol.sbc.org.br/index.php/ctrle/article/download/8870/8771',
 'https://sol.sbc.org.br/index.php/ctrle/article/download/8871/8772']

In [39]:
names_to_download = [el[-9:].replace("/", "-") for el in links_to_download]
names_to_download[:2]

['8870-8771', '8871-8772']

In [40]:
title = [remove_tab_br(el) for el in title]
authors = [remove_tab_br(el) for el in authors]

As função **download** e **transforma em texto** foram retiradas/inspiradas no material da aula.

A função worker tem como objetivo encapsular a tarefa de baixar e transformar para ser paraleliada.

In [41]:
import requests
from pdfminer.high_level import extract_text
import os


def download(url, nome):
    resposta = requests.get(url)
    if resposta.status_code == 200:
        with open(os.path.join("pdf", nome + ".pdf"), "wb") as f:
            f.write(resposta.content)


def remove_tab_br(string):
    return string.get_text().replace("\n", "").replace("\t", "")


def transforma_em_texto(nome):
    pdf_text = extract_text(os.path.join("pdf", nome + ".pdf"), codec="utf-8")
    with open(os.path.join("text", nome + ".txt"), "w", encoding="utf-8") as f:
        f.write(pdf_text)


def worker(i):
    link = links_to_download[i]
    name = names_to_download[i]
    download(link, name)
    transforma_em_texto(name)
    return f"Artigo {title[i]} pdf e txt gerado e salvo!"

Execução da celula seguir irá baixar paralelamente 75 artigos e transformar em texto. Os PDFs serão salvos no diretorio *pdf* e os arquivos txt no diretorio *txt*.

In [42]:
%%time
os.makedirs("pdf", exist_ok=True)
os.makedirs("text", exist_ok=True)

Parallel(verbose=1, n_jobs=10)(delayed(worker)(i) for i in range(len(names_to_download)))

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:   10.1s


Wall time: 21 s


[Parallel(n_jobs=10)]: Done  75 out of  75 | elapsed:   20.9s finished


['Artigo Como a Realidade Aumentada tem Auxiliado no Processo de Ensino e Aprendizagem de Ciências da Natureza? Um Mapeamento Sistemático da Literatura pdf e txt gerado e salvo!',
 'Artigo Investigação de Novas Estratégias para o Ensino de Bioquímica Estrutural por meio de Realidade Aumentada pdf e txt gerado e salvo!',
 'Artigo Modelagem Manual e Digital de Personagens como Ferramenta Lúdica e Auxiliar no Ensino-Aprendizagem de Anatomia Humana pdf e txt gerado e salvo!',
 'Artigo Utilização de Recursos Educacionais Digitais (RED) em Aulas de Língua Portuguesa nos Anos Finais do Ensino Fundamental: Contribuições e Desafios pdf e txt gerado e salvo!',
 'Artigo A Utilização de Softwares no Ensino de Matemática para Ensino Fundamental e Médio pdf e txt gerado e salvo!',
 'Artigo Virtualização de Jogos Matemáticos: uma Avaliação do Cubra Doze pdf e txt gerado e salvo!',
 'Artigo Metodologias Ativas à Luz de Comenius: uma Experiência na Pós-Graduação pdf e txt gerado e salvo!',
 'Artigo Lud

# Atividade 2: Coleta de Documentos em PDF